# McDonalds Diet Problem

In this notebook, we will determine the minimum cost McDonalds diet

Let's show how to read it from a spreadsheet
(mcdonalds.csv)

You need to take the mcdonalds.csv file in order to use this.

DataFrames is like Pandas in Python: https://dataframes.juliadata.org/stable/


In [3]:
using DataFrames, CSV, NamedArrays
df = CSV.read("mcdonalds.csv", DataFrame, delim =',') 

# Foods are column names starting at the 3rd
foods = propertynames(df)[3:end]

# Nutrients are the first column starting at the second row
nutrients = convert(Array, df[2:end,1])

# Cost
cost = Dict(zip(foods,df[1,3:end]))

required = Dict(zip(nutrients,df[2:end,2]))

# Matrix of data 
A = Matrix(df[2:end,3:end])

A_NA = NamedArray(A, (nutrients,foods), ("Nutrients", "Foods")) ;



In [5]:
using JuMP, HiGHS
m = Model(HiGHS.Optimizer)
@variable(m, x[foods] >= 0)
@objective(m, Min, sum(cost[j]*x[j] for j in foods))
@constraint(m, [i in nutrients], sum(A_NA[i,j]*x[j] for j in foods) >= required[i])

# If you don't want the solver output, you can try this
set_silent(m)
optimize!(m)

using Formatting
printfmtln("Minimum cost menu \${:.2f}: ", objective_value(m))
for j in foods
    if value(x[j]) > 1.0e-6
        printfmtln("Eat {1:.2f} of menu item {2}", value(x[j]), j)
    end
end

Minimum cost menu $14.86: 
Eat 4.39 of menu item Symbol("Quarter Pounder")
Eat 6.15 of menu item Symbol("Small Fries")
Eat 3.42 of menu item Symbol("1% Milk")
